In [2]:
# Importing the Necessary Libraries:

import sagemaker
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input, Session
import boto3

In [8]:
# Setting a Bucket Name:
bucket_name = 'bank-application-ml'
my_region = boto3.session.Session().region_name
print(my_region)

us-east-1


In [9]:
# Creating Bucket using code:
s3 = boto3.resource('s3')
try:
    if my_region == 'us-east-1':
        s3.create_bucket(Bucket = bucket_name)
    print('s3 bucket is created')
except Exception as e:
    print('s3 error: ', e)

s3 bucket is created


In [10]:
# We need an output path where the trained model will be saved:
prefix = 'xgboost-as-a-built-in-algo'
path = 's3://{}/{}/output'.format(bucket_name, prefix)
print(path)

s3://bank-application-ml/xgboost-as-a-built-in-algo/output


Now, It's time to download the bank dataset to train our model:

In [12]:
import pandas as pd
import urllib
try:
    urllib.request.urlretrieve ('https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv', 'bank_clean.csv')
    print('Success')
except Exception as e:
    print('Data Load Error: ', e)
try:
    df_model = pd.read_csv('./bank_clean.csv', index_col=0)
    print('Success: Data is Loaded')
except Exception as e:
    print('Data Load Error: ', e)

Success
Success: Data is Loaded


Next Step, is doing a Train-Test Split Using the NumPy Library:

In [13]:
import numpy as np
train, test = np.split(df_model.sample(frac=1, random_state=1729), [int(0.7 * len(df_model))])
print(train.shape, test.shape)

(28831, 61) (12357, 61)


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Now, Saving the Train and Test Datas into Buckets:

In [18]:
# Starting with the Train Data:
import os
from sagemaker.inputs import TrainingInput

pd.concat([train['y_yes'], train.drop(['y_no', 'y_yes'],
                                     axis = 1)],
         axis = 1).to_csv('train.csv', index = False, header = False)

# Uploading to S3 Bucket:
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = TrainingInput(s3_data = 's3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

In [19]:
# Now for the Test Data:
pd.concat([test['y_yes'], test.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('test.csv', index= False, header=False)

# Saving Test data to S3 Bucket
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = TrainingInput(s3_data = 's3://{}/{}/test'.format(bucket_name, prefix), content_type = 'csv')

Time to Build the XgBoost Model of Inbuilt Algorithm:

In [26]:
# Looking for the XGBoost Image URI:
region = boto3.Session().region_name
cont = sagemaker.image_uris.retrieve('xgboost',region,version = '1.0-1')
print(cont)

683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.0-1-cpu-py3


In [29]:
# Hyprparameters for XGBOost:
hyperparameters = {'max_depth': 5,
                  'eta':0.2,
                  'gamma':4,
                  'min_child_weight':6,
                  'subsample':0.7,
                  'objective':'binary:logistic',
                  'num_round':50}

In [34]:
# Constructing the Sagemaker Estimator:
estimator = sagemaker.estimator.Estimator(
    image_uri=cont,
    hyperparameters=hyperparameters,
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type='ml.m5.2xlarge',
    volume_size=5,  # Using 5GB
    output_path=path,
    use_spot_instances=True,
    max_run=300,
    max_wait=600
)

In [35]:
print (estimator)

In [36]:
estimator.fit({'train':s3_input_train, 'validation':s3_input_test})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-08-08-12-27-02-767


2024-08-08 12:27:02 Starting - Starting the training job...
2024-08-08 12:27:19 Starting - Preparing the instances for training...
2024-08-08 12:28:00 Downloading - Downloading the training image......
2024-08-08 12:29:01 Training - Training image download completed. Training in progress.
2024-08-08 12:29:01 Uploading - Uploading generated training model[2024-08-08 12:28:54.397 ip-10-2-72-172.ec2.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determ

Deploying the Machine Learning Model:

In [37]:
xgb_predictor = estimator.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-08-08-13-04-14-072
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2024-08-08-13-04-14-072
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2024-08-08-13-04-14-072


-------!

Prediction of Test Data:

In [39]:
from sagemaker.serializers import CSVSerializer


test_df_array = test.drop(['y_no', 'y_yes'], axis=1).values # we are loading the data to array

# Setting the data type for an inference:
xgb_predictor.content_type = 'text/csv'

# Serializer type:
xgb_predictor.serializer = CSVSerializer()

# Predicting:
preds = xgb_predictor.predict(test_df_array).decode('utf-8')

# Converting Prediction into an array:
preds_array = np.fromstring(preds[1:], sep=',')

print(preds_array.shape)

(12357,)


Creating a Confusion Matrix:

In [41]:
cm = pd.crosstab(index=test['y_yes'], columns = np.round(preds_array), rownames = ['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print('\n{0:<20}{1:<4.1f}%\n'.format('Overall Classification Rate: ', p))
print('{0:<15}{1:<15}{2:>8}'.format('Predicted', 'No Purchase', 'Purchase'))
print('Observed')
print('{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})'.format('No Purchase', tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print('{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n'.format('Purchase', fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.7%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10785)    34% (151)
Purchase        9% (1124)     66% (297) 



- The 'Purchase' and 'No Purchase' Accuracy of Observed and Predicted looks low of 9% probably due to the imbalance dataset.
- The 'Predicted' Value of Purchase and 'Observed' of 'No Purchase' gave out a 34%. We can increase the values by adjusting the hyperparameters 

FInally, I will delete the End Points so I can stop AWS from charging as soon as the training is done: 

In [42]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
delete_bucket = boto3.resource('s3').Bucket(bucket_name)
delete_bucket.objects.all().delete()

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker:Deleting endpoint with name: sagemaker-xgboost-2024-08-08-13-04-14-072


[{'ResponseMetadata': {'RequestId': 'QW2VWAVFKFBVZZPC',
   'HostId': '+d+H9CBEuxqKNFyEYz00bqzFFmJuxkpbwTaau2KMVoYr2kQB4b1Biag2LV63SpevtU1gY8l9VSM=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': '+d+H9CBEuxqKNFyEYz00bqzFFmJuxkpbwTaau2KMVoYr2kQB4b1Biag2LV63SpevtU1gY8l9VSM=',
    'x-amz-request-id': 'QW2VWAVFKFBVZZPC',
    'date': 'Thu, 08 Aug 2024 13:49:39 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2024-08-08-12-27-02-767/profiler-output/framework/training_job_end.ts'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2024-08-08-12-27-02-767/debug-output/index/000000000/000000000020_worker_0.json'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2024-08-08-12-27-02-767/debug-output/index/000000000/000000000000_worker_0.json'},
   {'Key': 'xgboost-as-a-bui